In [1]:
# Download packages
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
from pandas_datareader import data
import numpy as np

import pandas_datareader.data as web
import datetime as dt
import random


import gym
#import json
import datetime as dt
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import A2C
#from stable_baselines3.common.evaluation import evaluate_policy
import os
import matplotlib.pyplot as plt
from sklearn.utils import resample
import gym
from gym import spaces
	
from yahoo_fin.stock_info import get_analysts_info
import yahoo_fin.stock_info as si
#import optuna

import tensorflow as tf

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



c:\Users\F\AppData\Local\Programs\Python\Python37\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
# Download stock data



stocks = ['KINV-B.ST','TELIA.ST','NDA-SE.ST','SWMA.ST','TEL2-B.ST','ASSA-B.ST'
,'ERIC-B.ST','VOLV-B.ST','ELUX-B.ST','ABB.ST','ALIV-SDB.ST'
,'AZN.ST','SEB-A.ST','SKF-B.ST','SHB-A.ST'
,'ALFA.ST','INVE-B.ST','GETI-B.ST','HM-B.ST','SWED-A.ST'
,'HEXA-B.ST','ATCO-B.ST','SCA-B.ST','ATCO-A.ST','BOL.ST','SAND.ST']


# EVO.ST','ESSITY-B.ST',,'SINCH.ST','SBB-B.ST'
start='2010-01-01'
end='2015-01-01'
df = yf.download(stocks, 
                      start, 
                      end, 
                      progress=True)
df.reset_index(inplace=True)

omx = yf.download('^OMX', 
                      start, 
                      end, 
                      progress=True)
omx.reset_index(inplace=True)


omx = yf.download('^OMX', 
                      start, 
                      end, 
                      progress=True)
omx.reset_index(inplace=True)

omx = omx.loc[:,'Open']

Open=df.loc[:,'Open'].values
Close=df.loc[:,'Adj Close'].values
High=df.loc[:,'High'].values
Low=df.loc[:,'Low'].values
# Fix splits
#Open[630:,2] +=400

LOW=Low[1:,:]/Low[:-1,:]

HIGH=High[1:,:]/High[:-1,:]

OPEN = Open[1:,:]/Open[:-1,:]


#LOW=Low.iloc[1:,:].values
LOW= np.divide(LOW-np.min(LOW,axis=0),np.max(LOW,axis=0)-np.min(LOW,axis=0))

#HIGH=High.iloc[1:,:].values
HIGH= np.divide(HIGH-np.min(HIGH,axis=0),np.max(HIGH,axis=0)-np.min(HIGH,axis=0))

#OPEN = Open.iloc[1:,:].values

OPEN= np.divide(OPEN-np.min(OPEN,axis=0),np.max(OPEN,axis=0)-np.min(OPEN,axis=0))





x = pd.date_range(start, end,freq='D')

[*********************100%***********************]  26 of 26 completed

1 Failed download:
- SWMA.ST: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
# Environment


#Dimensions for data
frame_end = 15
Dim =10

Stocks = 26

class Test7():
    metadata = {'render.modes': ['human']}
    def __init__(self,LOW,HIGH,OPEN,Open,portfolio_start,frame_end,Stocks,Reward_function):


        self.open = OPEN
        self.low = LOW
        self.high = HIGH
        self.data = Open

        self.portfolio_start = portfolio_start
        self.Portfolio_balance = portfolio_start

        self.done = False

        self.shares_hold = 0
        self.fees = 0.001 # cost of buying stock

        self.total_sale = 0
        self.total_buy = 0
        self.total_fees = 0
        self.share_hold = 0
        self.share_hold_time = np.zeros((len(self.data),Stocks))

        self.buy_position = [0]*Stocks
        self.sell_position = [0]*Stocks
        self.reward_history = [0]*len(self.data)
        self.cash = self.portfolio_start
        self.stock_value = 0
        self.cash2stock_ratio = [0]*len(self.data)


       


        #self.frame_start = frame_start
        self.frame_end = frame_end



        # start data at 1 due to scaling
        self.tic = self.frame_end+1
        self.end = len(self.data)-self.frame_end-1

        self.net =[0]*len(self.data)
        self.sig_p =[0]*len(self.data)
        self.net.insert(self.tic,self.Portfolio_balance)
        
        self.Value_at_risk =[1]*len(self.data)

        self.plot_balance =[0]*len(self.data)
        self.plot_balance.insert(self.tic,self.Portfolio_balance)
        

        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(3,frame_end,Stocks), dtype=np.float32)                            
        self.action_space = spaces.Box(low = -1, high = 1,shape = (Stocks,)) 


    def Look_space(self):




        Look_1 = np.array([self.open[self.tic-self.frame_end: self.tic, :],
            self.low[self.tic-self.frame_end: self.tic,:],
             self.high[self.tic-self.frame_end: self.tic,:]])
    
            

       
            

        return Look_1

        
    def reset(self):
        self.done = False

        self.tic =self.frame_end+1

        self.reward = 0
        self.shares = 0
        self.share_hold_time = np.zeros((len(self.data),Stocks))

        self.Portfolio_balance = self.portfolio_start

        

        self.end = len(self.data)-self.frame_end-1


        self.total_sale = 0
        self.total_buy = 0
        self.total_fees = 0
        self.share_hold = [0]*Stocks
        
        self.cash = self.portfolio_start
        self.stock_value = 0
        self.cash2stock_ratio = [0]*len(self.data)


     
        self.buy_position = [0]*Stocks
        self.sell_position = [0]*Stocks
        self.reward_history = [0]*len(self.data)

        self.net =[0]*len(self.data)
        self.net.insert(self.tic,self.Portfolio_balance)

        self.Value_at_risk =[1]*len(self.data)

        self.plot_balance =[0]*len(self.data)
        self.sig_p =[0]*len(self.data)
        self.plot_balance.insert(self.tic,self.Portfolio_balance)

        self.frame_end = frame_end
        self.frame_start= frame_end

        state = self.Look_space()

        

        return state
    

    def buy_sell(self,action,index):
        

        

        # when does the stock buy? randomly?
        share_price =  self.data[self.tic, :]
        share_price1 = share_price[index]
        
        

        action_space = action
        amount =action*Dim
 
    


        if  action_space>0 and self.Portfolio_balance>0 :

            # Buy stocks
            P_buy =self.Portfolio_balance/(share_price1)
            amount_buy = min(amount,P_buy)
            buy = share_price1*amount_buy
            cost_penalty = buy*self.fees
            #self.buy_position.append(self.tic)



            self.Portfolio_balance -=cost_penalty+buy
            self.share_hold[index]+= amount_buy
            
           
            


        if action_space<0 and self.share_hold[index]>0:
            #Sell
            amount_sell = min(self.share_hold[index],abs(amount))
            #self.sell_position.append(self.tic)

            sold_amount = share_price1*amount_sell
            cost_penalty = sold_amount*self.fees

            self.Portfolio_balance +=sold_amount-cost_penalty

            self.share_hold[index] -= amount_sell

    def step(self,action):
        self.tic += 1
        a = np.array(action)


        index = 0
        for action in a:
            self.buy_sell(action,index)
            index +=1


        self.share_hold_time[self.tic,:] = self.share_hold

        self.stock_value = np.dot(self.share_hold,self.data[self.tic, :])
        self.cash = self.Portfolio_balance
        self.cash2stock_ratio.insert(self.tic,self.stock_value/self.cash)

        self.plot_balance.insert(self.tic,self.Portfolio_balance+np.dot(self.share_hold,self.data[self.tic, :]))
        R1 = np.divide(self.plot_balance[self.tic],self.plot_balance[self.tic-1])
        self.net.insert(self.tic,R1-1)
        matrix = np.corrcoef(self.data[self.tic-self.frame_end: self.tic, :],rowvar=False)
        weights =np.divide(np.multiply(self.data[self.tic, :],self.share_hold),self.plot_balance[self.tic])
        portfolio_vol = np.sqrt(np.dot(np.transpose(weights),np.dot(matrix,weights)))

        self.sig_p.insert(self.tic,portfolio_vol)


        if Reward_function==1:
 
            
            # Reward function Sharpe ratio
            R =np.divide(self.plot_balance[self.tic],self.plot_balance[self.tic-1])
            self.reward = np.divide(R,portfolio_vol)
            reward = np.divide(R-1,portfolio_vol)
        elif Reward_function==2:
            #Standard
            self.reward = (self.plot_balance[self.tic]-self.plot_balance[self.tic-1])
            reward = (self.reward /self.plot_balance[self.tic])
        elif Reward_function==3:
            R = (self.plot_balance[self.tic]/self.plot_balance[self.tic-1])
            self.Value_at_risk.insert(self.tic,R)
            K = np.sort(resample(self.Value_at_risk[self.tic-self.frame_end: self.tic],n_samples=20,replace=True))
            reward = R-0.5*K[2]
            

            
        
        self.reward_history.insert(self.tic,reward)

        obs = self.Look_space() 



        if self.tic >self.end :
            self.done = True
            self.Portfolio_balance += np.dot(self.share_hold,self.data[self.tic, :])*(1-self.fees)
            #self.render()
            

        


        return obs, reward, self.done, {}
    


        



TypeError: reset() missing 1 required positional argument: 'self'

In [ ]:
# training the model and saving it.
# Use tensorboard to evaluat the best model


log_path ="logs_netarch_0.001"
if not os.path.exists(log_path):
    os.makedirs(log_path)

for i in range(1,4):
    

    if i==1:

        Reward_function = 1
        Test = DummyVecEnv([lambda: Test7(LOW,HIGH,OPEN,Open,100000,frame_end,Stocks,Reward_function)])
        policy_kwargs = dict(act_fun=tf.nn.tanh, net_arch=[64, 64])
        model = PPO2(MlpPolicy, Test, verbose=1,policy_kwargs=policy_kwargs,tensorboard_log=log_path)
        models_dir ="models/Test_3_Sharp_64_64"
        if not os.path.exists( models_dir):
             os.makedirs( models_dir)
        Timestep = 10000
        for i in range(1,100):
            model.learn(total_timesteps=Timestep,reset_num_timesteps=False,tb_log_name="Test_3_Sharp_64_64")
            model.save(f"{models_dir}/{Timestep*i}")


        
        

    if i==2:
        Reward_function = 1
        Test = DummyVecEnv([lambda: Test7(LOW,HIGH,OPEN,Open,100000,frame_end,Stocks,Reward_function)])
        policy_kwargs = dict(act_fun=tf.nn.tanh, net_arch=[256, 256])
        model = PPO2(MlpPolicy, Test, verbose=1,policy_kwargs=policy_kwargs,tensorboard_log=log_path)
        models_dir ="models/Test_3_Sharp_256_256"
        if not os.path.exists(models_dir):
             os.makedirs(models_dir)
        Timestep = 10000
        for i in range(1,100):
            model.learn(total_timesteps=Timestep,reset_num_timesteps=False,tb_log_name="Test_3_Sharp_256_256")
            model.save(f"{models_dir}/{Timestep*i}")
        
    if i==3:
        Reward_function = 1
        Test = DummyVecEnv([lambda: Test7(LOW,HIGH,OPEN,Open,100000,frame_end,Stocks,Reward_function)])
        policy_kwargs = dict(act_fun=tf.nn.tanh, net_arch=[512, 512])
        model = PPO2(MlpPolicy, Test, verbose=1,policy_kwargs=policy_kwargs,tensorboard_log=log_path)
        models_dir ="models/Test_3_Sharp_512_512"
        if not os.path.exists(models_dir):
             os.makedirs(models_dir)
        Timestep = 10000
        for i in range(1,100):
            model.learn(total_timesteps=Timestep,reset_num_timesteps=False,tb_log_name="Test_3_Sharp_512_512")
            model.save(f"{models_dir}/{Timestep*i}")
      

 
 